In [1]:
from aocd import data, models, submit
from io import StringIO
from pathlib import Path
import re

import pandas as pd
import numpy as np
from itertools import product

# Load data and examples

In [2]:
puzzle_year = 2024
puzzle_day = int(re.match(r"day(\d+)", Path.cwd().name).group(1))

In [3]:
todays_puzzle = models.Puzzle(year=puzzle_year, day=puzzle_day)
data = todays_puzzle.input_data
todays_examples = todays_puzzle.examples

# Part A

Part B is much smarter...

In [4]:
cache = {}


def possible_control_codes_keypad_robot(
    code: str, keypad_coordinates: dict[str, tuple], avoid_position: tuple[int]
) -> list[str]:
    if (code, avoid_position) in cache:
        return cache[(code, avoid_position)]
    print(f"{code=}, {avoid_position=}")
    current_position = np.array(keypad_coordinates["A"])
    options = []
    for character in code:
        next_position = np.array(keypad_coordinates[character])
        diff = next_position - current_position
        vertical_moves = "".join(
            ["v"] * diff[0] if diff[0] > 0 else ["^"] * abs(diff[0])
        )
        horizontal_moves = "".join(
            [">"] * diff[1] if diff[1] > 0 else ["<"] * abs(diff[1])
        )
        current_possible_moves = []
        if np.any(current_position + (diff[0], 0) != avoid_position):
            current_possible_moves.append(vertical_moves + horizontal_moves + "A")
        if np.any(current_position + (0, diff[1]) != avoid_position):
            current_possible_moves.append(horizontal_moves + vertical_moves + "A")
        current_possible_moves = list(set(current_possible_moves))
        if len(options) == 0:
            options = current_possible_moves
        else:
            expand_options = []
            for option in options:
                for move in current_possible_moves:
                    expand_options.append(option + move)
            options = expand_options
        current_position = next_position
    cache[(code, avoid_position)] = options
    return options


def possible_control_codes_keypad_robot_multiple_codes(
    codes: list[str], keypad_coordinates: dict[str, tuple], avoid_position: tuple[int]
) -> list[str]:
    options = []
    for code in codes:
        options += possible_control_codes_keypad_robot(
            code, keypad_coordinates, avoid_position
        )
    return options

In [5]:
def part_a(data: str) -> str:
    directional_keypad_coordinates = {
        "^": (0, 1),
        "A": (0, 2),
        "<": (1, 0),
        "v": (1, 1),
        ">": (1, 2),
    }
    numerical_keypad_coordinates = {
        "7": (0, 0),
        "8": (0, 1),
        "9": (0, 2),
        "4": (1, 0),
        "5": (1, 1),
        "6": (1, 2),
        "1": (2, 0),
        "2": (2, 1),
        "3": (2, 2),
        "0": (3, 1),
        "A": (3, 2),
    }

    codes = data.splitlines()
    result = 0
    for code in codes:
        all_options = possible_control_codes_keypad_robot_multiple_codes(
            possible_control_codes_keypad_robot_multiple_codes(
                possible_control_codes_keypad_robot(
                    code, numerical_keypad_coordinates, (3, 0)
                ),
                directional_keypad_coordinates,
                (0, 0),
            ),
            directional_keypad_coordinates,
            (0, 0),
        )
        all_options = sorted(all_options, key=lambda x: len(x))
        min_length = len(all_options[0])
        code_numeric = int(re.match(r"([0-9]+)", "".join(code)).group(1))

        result += code_numeric * min_length
    return str(result)

In [ ]:
todays_examples[0] = todays_examples[0]._replace(
    input_data="""029A
980A
179A
456A
379A"""
)

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_a != "":
        print(
            f"Example {example_index} part a: {part_a(example.input_data)} (expected {example.answer_a})"
        )
        assert part_a(str(example.input_data)) == example.answer_a
submit(part_a(data), part="a", year=puzzle_year, day=puzzle_day)

# Part B

In [15]:
from itertools import permutations
from copy import deepcopy

In [16]:
def find_forbidden_path(
    start_position: tuple[int], end_position: tuple[int], forbidden_position: tuple[int]
) -> tuple[str]:
    diff_forbidden_start = np.array(forbidden_position) - np.array(start_position)
    diff_forbidden_end = np.array(end_position) - np.array(forbidden_position)
    result = []
    if any(diff_forbidden_start == 0) and any(diff_forbidden_end == 0):
        result += (
            ["v"] * diff_forbidden_start[0]
            if diff_forbidden_start[0] > 0
            else ["^"] * abs(diff_forbidden_start[0])
        )
        result += (
            [">"] * diff_forbidden_start[1]
            if diff_forbidden_start[1] > 0
            else ["<"] * abs(diff_forbidden_start[1])
        )
        result += (
            ["v"] * diff_forbidden_end[0]
            if diff_forbidden_end[0] > 0
            else ["^"] * abs(diff_forbidden_end[0])
        )
        result += (
            [">"] * diff_forbidden_end[1]
            if diff_forbidden_end[1] > 0
            else ["<"] * abs(diff_forbidden_end[1])
        )
    return tuple(result)


def minimal_step_change_generic_keypad(
    keypad_control: dict[str, tuple[int]],
    forbidden_point: tuple[int],
    start_button: str,
    button_to_press: str,
    pairwise_cost: dict[tuple[str, str], int],
) -> int:
    forbidden_path = find_forbidden_path(
        keypad_control[start_button], keypad_control[button_to_press], forbidden_point
    )
    diff = np.array(keypad_control[button_to_press]) - np.array(
        keypad_control[start_button]
    )
    vertical_moves = "".join(["v"] * diff[0] if diff[0] > 0 else ["^"] * abs(diff[0]))
    horizontal_moves = "".join([">"] * diff[1] if diff[1] > 0 else ["<"] * abs(diff[1]))
    unique_moves = set(permutations(vertical_moves + horizontal_moves)) - set(
        [forbidden_path]
    )
    min_cost = float("inf") if len(unique_moves) > 0 else pairwise_cost[("A", "A")]
    for move in unique_moves:
        cost = 0
        if len(move) > 0:
            cost += pairwise_cost[("A", move[0])]
            cost += pairwise_cost[(move[-1], "A")]
        else:
            cost += pairwise_cost[("A", "A")]
        for i in range(len(move) - 1):
            cost += pairwise_cost[(move[i], move[i + 1])]
        min_cost = min(min_cost, cost)
    return min_cost


def minimal_steps_change_directional_keypad_robot(
    start_button: str, button_to_press: str, pairwise_cost: dict[tuple[str, str], int]
) -> int:
    keypad_control = {"^": (0, 1), "A": (0, 2), "<": (1, 0), "v": (1, 1), ">": (1, 2)}
    forbidden_point = (0, 0)
    return minimal_step_change_generic_keypad(
        keypad_control, forbidden_point, start_button, button_to_press, pairwise_cost
    )


def minimal_step_change_numerical_keypad_robot(
    start_button: str, button_to_press: str, pairwise_cost: dict[tuple[str, str], int]
) -> int:
    keypad_coordinates = {
        "7": (0, 0),
        "8": (0, 1),
        "9": (0, 2),
        "4": (1, 0),
        "5": (1, 1),
        "6": (1, 2),
        "1": (2, 0),
        "2": (2, 1),
        "3": (2, 2),
        "0": (3, 1),
        "A": (3, 2),
    }
    forbidden_point = (3, 0)
    return minimal_step_change_generic_keypad(
        keypad_coordinates,
        forbidden_point,
        start_button,
        button_to_press,
        pairwise_cost,
    )

In [17]:
def get_minimal_steps_to_press_code(code: str, num_directional_robots: int):
    directional_buttons = ["^", "<", "v", ">", "A"]
    # The cost of the human pressing any buttons is constant
    initial_cost = {(x, y): 1 for x, y in product(directional_buttons, repeat=2)}
    pairwise_cost = deepcopy(initial_cost)
    for _ in range(num_directional_robots):
        new_pairwise_cost = {}
        for start_button in directional_buttons:
            for button_to_press in directional_buttons:
                new_pairwise_cost[(start_button, button_to_press)] = (
                    minimal_steps_change_directional_keypad_robot(
                        start_button, button_to_press, pairwise_cost
                    )
                )
        pairwise_cost = new_pairwise_cost
    code_for_loop = "A" + code
    cost = 0
    for i in range(len(code_for_loop) - 1):
        cost += minimal_step_change_numerical_keypad_robot(
            code_for_loop[i], code_for_loop[i + 1], pairwise_cost
        )
    return cost

In [18]:
def part_b(data: str, num_directional_robots: int = 25) -> str:
    codes = data.splitlines()
    result = 0
    for code in codes:
        code_execution_length = get_minimal_steps_to_press_code(
            code, num_directional_robots
        )
        code_numeric = int(re.match(r"([0-9]+)", "".join(code)).group(1))
        result += code_numeric * code_execution_length
    return str(result)

In [ ]:
assert part_a(data) == part_b(data, 2)
submit(part_b(data), part="b", year=puzzle_year, day=puzzle_day)